In [20]:
from neo4j import GraphDatabase
import DATABASE_CONFIG

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", DATABASE_CONFIG.DB_PASSWORD)

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "CREATE CONSTRAINT id_desmatamento_constraint FOR (r:Desmatamento) REQUIRE r.id_desmatamento IS UNIQUE"
    )

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'file:///relatorio_desmatamento.csv' AS csvLine "
        "CREATE (r:Desmatamento {id_desmatamento: toInteger(csvLine.id_desmatamento), area_km2: toFloat(csvLine.area_km2), " \
        "ano: toInteger(csvLine.ano)})"
    )

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        """
        LOAD CSV WITH HEADERS FROM 'file:///relatorio_desmatamento.csv' AS csvLine
        MATCH (d:Desmatamento {id_desmatamento: toInteger(csvLine.id_desmatamento)})

        CALL {
            WITH d, csvLine
            OPTIONAL MATCH (a:AreaIndigena {id_area_indigena: toInteger(csvLine.id_area_indigena)})
            WHERE a IS NOT NULL
            FOREACH (_ IN CASE WHEN a IS NOT NULL THEN [1] ELSE [] END |
                MERGE (d)-[:Ocorreu_Em]->(a)
            )
        }

        CALL {
            WITH d, csvLine
            OPTIONAL MATCH (c:Cidade {id_cidade: toInteger(csvLine.id_cidade)})
            WHERE c IS NOT NULL
            FOREACH (_ IN CASE WHEN c IS NOT NULL THEN [1] ELSE [] END |
                MERGE (d)-[:Ocorreu_Em]->(c)
            )
        }

        CALL {
            WITH d, csvLine
            OPTIONAL MATCH (u:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.id_unidade_conservacao)})
            WHERE u IS NOT NULL
            FOREACH (_ IN CASE WHEN u IS NOT NULL THEN [1] ELSE [] END |
                MERGE (d)-[:Ocorreu_Em]->(u)
            )
        }
        """
        )

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (d, csvLine) { ... }} {position: line: 5, column: 9, offset: 187} for query: "\n        LOAD CSV WITH HEADERS FROM 'file:///relatorio_desmatamento.csv' AS csvLine\n        MATCH (d:RelatorioDesmatamento {id_desmatamento: toInteger(csvLine.id_desmatamento)})\n\n        CALL {\n            WITH d, csvLine\n            OPTIONAL MATCH (a:AreaIndigena {id_area_indigena: toInteger(csvLine.id_area_indigena)})\n            WHERE a IS NOT NULL\n            FOREACH (_ IN CASE WHEN a IS NOT NULL THEN [1] ELSE [] END |\n                MERGE (d)-[:Ocorreu_Em]->(a)\n            )\n        }\n\n        CALL {\n            WITH d, csvLine\n            OPTIONAL MATCH (c:Cidade {id_cidade: toI